In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import joblib

# Load the model (place chronic_model.pkl in same folder)
model = joblib.load("chronic_model.pkl")

# Create input fields
glucose = widgets.FloatText(description="Glucose")
bp = widgets.FloatText(description="BP")
bmi = widgets.FloatText(description="BMI")
age = widgets.IntText(description="Age")
button = widgets.Button(description="Predict", button_style="success")
output = widgets.Output()

def on_click(b):
    with output:
        output.clear_output()
        df = pd.DataFrame([{
            "glucose": glucose.value,
            "bp": bp.value,
            "bmi": bmi.value,
            "age": age.value
        }])
        prediction = model.predict(df)[0]
        print(f"🧠 Risk Prediction: {prediction}")

button.on_click(on_click)

display(glucose, bp, bmi, age, button, output)
